In [1]:
import os
import sagemaker
#from sagemaker import get_execution_role
print(sagemaker.__version__)

sagemaker_session = sagemaker.Session()

#role = get_execution_role()
role = 'arn:aws:iam::776620588530:role/service-role/AmazonSageMaker-ExecutionRole-20210423T144098'

2.38.0


In [2]:
role

'arn:aws:iam::776620588530:role/service-role/AmazonSageMaker-ExecutionRole-20210423T144098'

In [3]:
region = sagemaker_session.boto_session.region_name

region

'eu-west-2'

In [4]:
import os
import keras
import numpy as np
from keras.datasets import fashion_mnist

(x_train, y_train), (x_val, y_val) = fashion_mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)

In [4]:
#training_data_uri = 's3://sagemaker-sample-data-{}/tensorflow/mnist'.format(region)

In [5]:
from sagemaker.tensorflow import TensorFlow

In [7]:
# Train on local data. S3 URIs would work too.
training_input_path   = 'file://data/training.npz'
validation_input_path = 'file://data/validation.npz'

# Store model locally. A S3 URI would work too.
output_path           = 'file:///tmp/model/'

In [9]:
mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             instance_count=1,
                             instance_type='local',
                             framework_version='1.12',
                             py_version = 'py3',
                             output_path = output_path
                            )

In [10]:
mnist_estimator.fit(training_input_path)

FileNotFoundError: [Errno 2] No such file or directory: 'mnist.py'

In [12]:
predictor = mnist_estimator.deploy(initial_instance_count=1, 
                                   instance_type='ml.p3.2xlarge',
                                   endpoint_name='image-classification-2019-09-03-05-58-51-313')

-------------------------------------------------------------------------------------!

In [13]:
import numpy as np

get_ipython().system('aws --region {region} s3 \
                     cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_data.npy train_data.npy')

get_ipython().system('aws --region {region} s3 \
                     cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_labels.npy train_labels.npy')

download: s3://sagemaker-sample-data-us-east-2/tensorflow/mnist/train_data.npy to ./train_data.npy
download: s3://sagemaker-sample-data-us-east-2/tensorflow/mnist/train_labels.npy to ./train_labels.npy


In [14]:
train_data = np.load('train_data.npy')
train_labels = np.load('train_labels.npy')

In [15]:
predictions = predictor.predict(train_data[:50])

In [16]:
predictions

{'predictions': [{'probabilities': [1.80712e-05,
    6.39894e-05,
    0.00775659,
    0.170431,
    4.68211e-05,
    9.73051e-05,
    1.12912e-06,
    0.724196,
    0.0014382,
    0.0959506],
   'classes': 7},
  {'probabilities': [6.56517e-07,
    4.34871e-07,
    7.03569e-05,
    0.997957,
    1.73637e-08,
    0.000278024,
    1.31928e-09,
    5.45352e-07,
    0.00121957,
    0.000473886],
   'classes': 3},
  {'probabilities': [1.1415e-06,
    3.30415e-06,
    2.44502e-06,
    8.65166e-05,
    0.635296,
    0.000732875,
    3.43277e-06,
    0.00414868,
    0.00177787,
    0.357948],
   'classes': 4},
  {'probabilities': [0.000224655,
    8.24956e-06,
    0.00254671,
    1.7955e-06,
    0.00109289,
    0.000581131,
    0.995533,
    5.95401e-07,
    3.33052e-06,
    7.26739e-06],
   'classes': 6},
  {'probabilities': [1.02781e-05,
    0.997066,
    0.000640446,
    9.79578e-05,
    0.000121962,
    6.84923e-05,
    0.000597569,
    0.000163621,
    0.0012148,
    1.85635e-05],
   'clas

In [17]:
for i in range(0, 50):
    prediction = predictions['predictions'][i]['classes']
    label = train_labels[i]
    print('prediction is {}, label is {}, matched: {}'.format(prediction, label, prediction == label))

prediction is 7, label is 7, matched: True
prediction is 3, label is 3, matched: True
prediction is 4, label is 4, matched: True
prediction is 6, label is 6, matched: True
prediction is 1, label is 1, matched: True
prediction is 8, label is 8, matched: True
prediction is 1, label is 1, matched: True
prediction is 0, label is 0, matched: True
prediction is 9, label is 9, matched: True
prediction is 8, label is 8, matched: True
prediction is 0, label is 0, matched: True
prediction is 3, label is 3, matched: True
prediction is 1, label is 1, matched: True
prediction is 2, label is 2, matched: True
prediction is 7, label is 7, matched: True
prediction is 0, label is 0, matched: True
prediction is 2, label is 2, matched: True
prediction is 9, label is 9, matched: True
prediction is 6, label is 6, matched: True
prediction is 0, label is 0, matched: True
prediction is 1, label is 1, matched: True
prediction is 6, label is 6, matched: True
prediction is 7, label is 7, matched: True
prediction 